In [4]:
from __future__ import division
import pdb 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import util
import seaborn as sns 
import clean_data as cd
import benchmark
import group_models

In [2]:
pd2007 = cd.import_data(2007)
pd2012 = cd.import_data(2012)

all_data = pd.concat([pd2007, pd2012])
prepped_data = cd.prep_for_classification(all_data)
X, y = cd.feature_target_split(prepped_data)

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
# grades are still here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
train_payouts = all_data.ix[X_train.index]['payout_prop']
test_payouts = all_data.ix[X_test.index]['payout_prop']

print "Benchmark for all data on A/B/C: %.2f" %benchmark.total_return(all_data, filter_col='grade', filter_vals=['A', 'B', 'C'])
print "Benchmar for all data on all loans: %.2f" %benchmark.total_return(all_data)
print "Benchmark for all data is : %.2f" %np.mean(all_data['payout_prop'])
print "Benchmark for 2012 data is: %.2f" %np.mean(pd2012['payout_prop'])
print "Benchmark for 2007 data is: %.2f" %np.mean(pd2007['payout_prop'])

lr_params = { 
    'C': 100,
    'class_weight': {False: 15},
    'penalty': 'l2',
    'n_jobs': -1
}

models = group_models.fit(X_train, y_train, 'grade', LogisticRegression, lr_params)
group_models.group_results(X_test, y_test, test_payouts, 'grade', models)

Benchmark for all data on A/B/C: 1.08
Benchmar for all data on all loans: 1.08
Benchmark for all data is : 1.08
Benchmark for 2012 data is: 1.09
Benchmark for 2007 data is: 1.08
Making group for group A
Making group for group B
Making group for group C
Making group for group D
Making group for group E
Making group for group F
Making group for group G
Precision for group A: 0.90
Benchmark for group A: 1.09
[[ 349   32]
 [  40 5995]]
Precision for group B: 0.77
Benchmark for group B: 1.16
[[1251  114]
 [ 376 9304]]
Precision for group C: 0.18
Benchmark for group C: 1.03
[[1212    5]
 [5598   13]]
Precision for group D: 0.23
Benchmark for group D: 1.15
[[ 848    0]
 [2855    1]]
Precision for group E: 0.24
Benchmark for group E: 0.58
[[187   2]
 [582   3]]
Precision for group F: 0.34
Benchmark for group F: nan
[[41  0]
 [79  0]]
Precision for group G: 0.27
Benchmark for group G: nan
[[ 4  0]
 [11  0]]
Overall precision: 3892 9541
Overall benchmark: 1.13


In [8]:
all_lr_model = LogisticRegression(**lr_params)
all_lr_model.fit(X_train.drop('grade', axis=1), y_train)

LogisticRegression(C=100, class_weight={False: 15}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [10]:
predicted = all_lr_model.predict(X_test.drop('grade', axis=1))
cm_all_model = confusion_matrix(y_test, predicted, labels=[True, False])

In [13]:
precision_all = cm_all_model[0,0] / np.sum(cm_all_model[:, 0])
print precision_all

0.987076417704


In [14]:
all_model_indices = [X_test.index[i] for i in range(len(X_test)) if predicted[i] == 1]
predicted_df = all_data.ix[all_model_indices]
benchmark.total_return(predicted_df)

1.1659078177304181

In [15]:
print cm_all_model

[[22837  2020]
 [  299  3746]]


In [25]:
test_copy = X_test.copy()
test_copy['prediction'] = predicted
grouped_test = test_copy.groupby('grade')

for gname, df in grouped_test:
    y_group = y_test.ix[df.index]
    print (" Total pred 1: ", np.sum(df['prediction']))
    cm_group = confusion_matrix(y_group, df['prediction'], labels=[True, False])
    print "Group %s" %gname
    print cm_group
    print "Precision: %.2f" %(float(cm_group[0, 0]) / float(cm_group[0,0] + cm_group[1, 0]))
    print("=======================================================")
    # grab rows by grade
    

(' Total pred 1: ', 6011)
Group A
[[5965   70]
 [  46  335]]
Precision: 0.99
(' Total pred 1: ', 9438)
Group B
[[9312  368]
 [ 126 1239]]
Precision: 0.99
(' Total pred 1: ', 4980)
Group C
[[4902  709]
 [  78 1139]]
Precision: 0.98
(' Total pred 1: ', 2249)
Group D
[[2210  646]
 [  39  809]]
Precision: 0.98
(' Total pred 1: ', 399)
Group E
[[391 194]
 [  8 181]]
Precision: 0.98
(' Total pred 1: ', 52)
Group F
[[50 29]
 [ 2 39]]
Precision: 0.96
(' Total pred 1: ', 7)
Group G
[[7 4]
 [0 4]]
Precision: 1.00
